In [1]:
import jax
import jax.numpy as jnp
import immrax as irx

In [9]:
def f (x) :
    return x - x

jit_f = jax.jit(f)
jit_f (0.)
print(jax.make_jaxpr(f)(0.))
print(jax.make_jaxpr(jit_f)(0.))
print(jax.make_jaxpr(irx.natif(f))(irx.interval(0.)))
print(irx.natif(f)(irx.interval(0., 1.)))
print(jax.make_jaxpr(irx.jacif(f))(irx.interval(0.)))
print(irx.jacif(f)(irx.interval(0., 1.)))

{ lambda ; a:f32[]. let b:f32[] = sub a a in (b,) }
{ lambda ; a:f32[]. let
    b:f32[] = pjit[
      name=f
      jaxpr={ lambda ; c:f32[]. let d:f32[] = sub c c in (d,) }
    ] a
  in (b,) }
{ lambda ; a:f32[] b:f32[]. let
    c:f32[] d:f32[] = pjit[
      name=f
      jaxpr={ lambda ; e:f32[] f:f32[]. let
          g:f32[] = sub e f
          h:f32[] = sub f e
        in (g, h) }
    ] a b
  in (c, d) }
-1.0 <= x <= 1.0
let atleast_1d = { lambda ; a:f32[]. let
    b:f32[1] = reshape[dimensions=None new_sizes=(1,)] a
  in (b,) } in
{ lambda ; c:f32[] d:f32[]. let
    e:f32[1] f:f32[1] = pjit[
      name=F
      jaxpr={ lambda ; g:f32[] h:f32[]. let
          i:f32[1] = pjit[name=atleast_1d jaxpr=atleast_1d] g
          j:f32[1] = pjit[name=atleast_1d jaxpr=atleast_1d] h
          k:f32[1] = add i j
          l:f32[1] = div k 2.0
          m:f32[1,1] = pjit[
            name=f
            jaxpr={ lambda ; n:f32[1] o:f32[1]. let
                _:f32[1] = sub n o
                _:f32[